# 실습 6: Agent Runtime 배포

## 목표
- bedrock_agent_core.py 생성
- requirements.txt 생성
- Bedrock Agent Core Runtime 배포

## 6-1. 설정 확인

In [ ]:
from workshop_config import REGION
print(f"🌍 Region: {REGION}")

## 6-2. bedrock_agent_core.py 생성

In [ ]:
%%writefile agent/deploy/runtime/bedrock_agent_core.py
"""Bedrock AgentCore Runtime Entrypoint"""

from bedrock_agentcore.runtime import BedrockAgentCoreApp
from agent.factory import create_agent

app = BedrockAgentCoreApp()

@app.entrypoint
async def integrated_disaster_agent(payload):
    """Handler for agent invocation"""
    user_message = payload.get("prompt", "Hello!")
    actor_id = payload.get("actor_id", "default-user")
    session_id = payload.get("session_id", "default-session")
    
    print(f"Processing: actor={actor_id}, session={session_id}")
    
    agent = create_agent(actor_id, session_id)
    
    stream = agent.stream_async(user_message)
    async for event in stream:
        yield event

if __name__ == "__main__":
    app.run()

## 6-3. requirements.txt 생성

In [ ]:
%%writefile agent/deploy/runtime/requirements.txt
strands-agents>=0.5.0
strands-agents-tools>=0.1.0
boto3>=1.34.0
bedrock-agentcore-starter-toolkit
bedrock-agentcore
wikipedia
playwright>=1.40.0
nest-asyncio

## 6-4. workshop_config.py 최종 동기화

In [ ]:
import shutil
shutil.copy('workshop_config.py', 'agent/deploy/runtime/')
print("✅ workshop_config.py 동기화 완료")

## 6-5. Runtime 배포

In [ ]:
import os
os.chdir('agent/deploy/runtime')
print(f"📂 현재 디렉토리: {os.getcwd()}")

In [ ]:
from workshop_config import REGION
from bedrock_agentcore_starter_toolkit import Runtime

print("🚀 Agent Runtime 배포 시작...\n")

agentcore_runtime = Runtime()

print("1️⃣ Runtime 설정 중...")
agentcore_runtime.configure(
    entrypoint="bedrock_agent_core.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=REGION,
    agent_name="agent_runtime"
)

print("\n2️⃣ Docker 빌드 및 배포 중... (5-10분)\n")
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

print("\n✅ 배포 완료!")
print(f"📝 Agent ARN: {launch_result.agent_arn}")

AGENT_RUNTIME_ARN = launch_result.agent_arn

## 6-6. workshop_config.py 업데이트

In [ ]:
import os
os.chdir('../../..')  # 프로젝트 루트로 이동

# workshop_config.py 업데이트
with open('workshop_config.py', 'r') as f:
    content = f.read()

content = content.replace('AGENT_RUNTIME_ARN = ""', f'AGENT_RUNTIME_ARN = "{AGENT_RUNTIME_ARN}"')

with open('workshop_config.py', 'w') as f:
    f.write(content)

print(f"✅ workshop_config.py 업데이트 완료")
print(f"📝 ARN: {AGENT_RUNTIME_ARN}")

## ✅ 실습 6 완료!

### 생성된 리소스
- Agent Runtime (Bedrock Agent Core)
- ECR Repository
- Docker Image
- Execution Role

### 생성된 파일
- `agent/deploy/runtime/bedrock_agent_core.py`
- `agent/deploy/runtime/requirements.txt`

### 업데이트된 파일
- `workshop_config.py` (AGENT_RUNTIME_ARN 추가)

👉 다음: **lab7_web_application.ipynb**